# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Get results from first part
vac_df=pd.read_csv('outputfile_ras.csv')
# Drop any rowswith NaN
vac_df=vac_df.dropna()

# Look at header for column names
vac_df.head()

,Unnamed: 0,city,id,lat,temp,humidity,clouds,wind,lng,Country
0,0,Ardakān,143073,32.31,68.00,33,40,13.87,54.02,IR
1,1,Ushuaia,3833367,-54.80,55.40,47,20,6.93,-68.30,AR
2,2,Cañon City,5416005,38.44,39.11,80,90,6.98,-105.24,US
3,3,Aklavik,5882953,68.22,23.70,67,90,9.17,-135.01,CA
4,4,Punta Arenas,3874787,-53.15,59.00,58,75,20.80,-70.92,CL


In [3]:
# Select columns desired

vac_df=vac_df[['city','lat','lng','temp','humidity','wind','clouds','Country']]
vac_df.head()



,city,lat,lng,temp,humidity,wind,clouds,Country
0,Ardakān,32.31,54.02,68.00,33,13.87,40,IR
1,Ushuaia,-54.80,-68.30,55.40,47,6.93,20,AR
2,Cañon City,38.44,-105.24,39.11,80,6.98,90,US
3,Aklavik,68.22,-135.01,23.70,67,9.17,90,CA
4,Punta Arenas,-53.15,-70.92,59.00,58,20.80,75,CL


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
import gmaps
gmaps.configure(api_key=g_key)


In [5]:
locations=vac_df[['lat','lng']].astype(float)

wt=vac_df['humidity'].astype(float)

fig=gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=wt, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Wind <=10 mph
vac1_df=vac_df[vac_df['wind']<=10]
# Temp 45-85
vac1_df=vac1_df[vac1_df['temp']<=85]
vac1_df=vac1_df[vac1_df['temp']>=45]
# clouds<=20%, because I like the stars/astronomy
vac1_df=vac1_df[vac1_df['clouds']<=20]
#humidity <= 50%
vac1_df=vac1_df[vac1_df['humidity']<=50]
vac1_df=vac1_df.dropna()
len(vac1_df)

13

# Guessing this will put me in the desert, which is a good place to star gaze

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

# Sentence fragment.  What is the object of the verb 'store'? I have no idea. The instructions for this assignment and the examples provided are not very helpful. (But I cannot imagine ever needing this in what I do.) Why hit Google places for something already in the dataframe (coords)? (This assignment does not make sense.)

In [7]:
dex=np.arange(0,len(vac1_df.index))
hotel1_df=pd.DataFrame(vac1_df)
city_list=vac1_df['city'].tolist()
hotel_df=pd.DataFrame({'city':city_list},index=dex)
hotel_df['city']=vac1_df['city'].tolist()
hotel_df['lat']=vac1_df['lat'].tolist()
hotel_df['lng']=vac1_df['lng'].tolist()
hotel_df['Country']=vac1_df['Country'].tolist()
hotel_df['Hotel Name']=''
hotel_df.head()

,city,lat,lng,Country,Hotel Name
0,Ushuaia,-54.80,-68.30,AR,
1,Nālūt,30.33,10.85,LY,
2,Atar,20.52,-13.05,MR,
3,Taoudenni,22.68,-3.98,ML,
4,Nanzhang,31.78,111.84,CN,


In [8]:
import json
hotel_list=[]
locations=[]
for index, row in hotel_df.iterrows():
    lat1=hotel_df.iloc[index,1]
    long1=hotel_df.iloc[index,2]
    coord=f"{lat1}, {long1}"
    locations.append(coord)
    url=f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
    req=f"input=lodging&inputtype=textquery&fields=name,rating&locationbias=circle:5000@{coord}&key={g_key}"
    response=requests.get(url+req)
    response_json=response.json()
    hotel_list.append(response_json['candidates'][0]['name'])
    
hotel_df['Hotel Name']=hotel_list
hotel_df.head(10)

,city,lat,lng,Country,Hotel Name
0,Ushuaia,-54.80,-68.30,AR,Los Cauquenes Resort & Spa
1,Nālūt,30.33,10.85,LY,HHC Hotels LLC
2,Atar,20.52,-13.05,MR,Odar kanawal
3,Taoudenni,22.68,-3.98,ML,HHC Hotels LLC
4,Nanzhang,31.78,111.84,CN,南漳青豪旅馆
5,Ures,29.43,-110.40,MX,Hacienda del Labrador
6,Danielskuil,-28.19,23.54,ZA,Old Post Guest House
7,Bariloche,-41.15,-71.31,AR,La Justina
8,Changping,40.22,116.22,CN,Beijing Marriott Hotel Changping
9,Adrar,20.50,-10.07,MR,HHC Hotels LLC


In [9]:
hotel_list1=hotel_df["Hotel Name"]
hotel_list1

0           Los Cauquenes Resort & Spa
1                       HHC Hotels LLC
2                         Odar kanawal
3                       HHC Hotels LLC
4                               南漳青豪旅馆
5                Hacienda del Labrador
6                 Old Post Guest House
7                           La Justina
8     Beijing Marriott Hotel Changping
9                       HHC Hotels LLC
10          Expeditions Ténére voyages
11                  Vivanta Aurangabad
12                    West Texas Lodge
Name: Hotel Name, dtype: object

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [11]:
# Add marker layer ontop of heat map
locations=hotel_df[['lat','lng']]
marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel : {hotel}" for hotel in hotel_list1]
)

fig=gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))